In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Reload the CSV files into pandas DataFrames for years 2018, 2019, and 2020
df_2018 = pd.read_csv('ITSalary2018.csv')
df_2019 = pd.read_csv('ITSalary2019.csv')
df_2020 = pd.read_csv('ITSalary2020.csv')

In [3]:
# Concatenate the three dataframes along the rows
df = pd.concat([df_2018, df_2019, df_2020], ignore_index=True)

In [4]:
# Check columns 
df.columns.to_list()

['Timestamp',
 'Age',
 'Gender',
 'City',
 'Position',
 'Years of experience',
 'Your level',
 'Current Salary',
 'Salary one year ago',
 'Salary two years ago',
 'Are you getting any Stock Options?',
 'Main language at work',
 'Company size',
 'Company type',
 'Zeitstempel',
 'Seniority level',
 'Position (without seniority)',
 'Your main technology / programming language',
 'Yearly brutto salary (without bonus and stocks)',
 'Yearly bonus',
 'Yearly stocks',
 'Yearly brutto salary (without bonus and stocks) one year ago. Only answer if staying in same country',
 'Yearly bonus one year ago. Only answer if staying in same country',
 'Yearly stocks one year ago. Only answer if staying in same country',
 'Number of vacation days',
 'Number of home office days per month',
 'Company name ',
 'Сontract duration',
 'Company business sector',
 '0',
 'Total years of experience',
 'Years of experience in Germany',
 'Other technologies/programming languages you use often',
 'Yearly brutto salary

# Data Processing

In [5]:
# Keep only attributes that can be found in Job_Listing Project Dataframe,
# notice that every dataframe had a different name for salaries
columns_to_keep = ["City", 
                   "Position",
                   "Position (without seniority)",
                   "Main language at work", 
                   "Seniority level",
                   "Yearly brutto salary (without bonus and stocks)" , 
                   "Yearly brutto salary (without bonus and stocks) in EUR", 
                   "Current Salary"]

In [6]:
# Create new dataframe with selected columns
filtered_df = df[columns_to_keep].copy()  # Create a copy of the DataFrame to prevent warning

In [7]:
filtered_df.head()

,City,Position,Position (without seniority),Main language at work,Seniority level,Yearly brutto salary (without bonus and stocks),Yearly brutto salary (without bonus and stocks) in EUR,Current Salary
0,München,QA Ingenieur,NaN,Deutsch,NaN,NaN,NaN,77000.0
1,München,Senior PHP Magento developer,NaN,Deutsch,NaN,NaN,NaN,65000.0
2,München,Software Engineer,NaN,Deutsch,NaN,NaN,NaN,88000.0
3,München,Senior Frontend Developer,NaN,English,NaN,NaN,NaN,78000.0
4,München,UX Designer,NaN,English,NaN,NaN,NaN,69000.0


In [8]:
# Merge the 3 different salaries columns 
filtered_df['Current Salary'] = (filtered_df['Current Salary'].astype(str) +
                               filtered_df['Yearly brutto salary (without bonus and stocks) in EUR'].astype(str) +
                               filtered_df['Yearly brutto salary (without bonus and stocks)'].astype(str))

In [9]:
# Merge the 2 different position columns 
filtered_df['Position'] = (filtered_df['Position'].astype(str) +
                           filtered_df['Position (without seniority)'].astype(str))

In [10]:
# Drop the original columns as they're now redundant
filtered_df = filtered_df.drop(['Yearly brutto salary (without bonus and stocks) in EUR', 
                                'Yearly brutto salary (without bonus and stocks)',
                                'Position (without seniority)' ], axis=1)


In [11]:
# Rename the columns based on the provided mappings
filtered_df = filtered_df.rename(columns={
    'City': 'location',
    'Position': 'title',
    'Main language at work': 'language',
    'Seniority level': 'level',
    'Current Salary': 'salary'
})

In [12]:
filtered_df.head()

,location,title,language,level,salary
0,München,QA Ingenieurnan,Deutsch,NaN,77000.0nannan
1,München,Senior PHP Magento developer nan,Deutsch,NaN,65000.0nannan
2,München,Software Engineernan,Deutsch,NaN,88000.0nannan
3,München,Senior Frontend Developernan,English,NaN,78000.0nannan
4,München,UX Designernan,English,NaN,69000.0nannan


# Location cleanup

In [13]:
# Inspect location
print(f'Number of location missing values: {filtered_df.location.isna().sum()}')
filtered_df['location'].value_counts()

Number of location missing values: 29


location
Berlin            1402
Munich             476
München            249
Frankfurt          127
Amsterdam          104
                  ... 
Frankonia            1
Kaiserslautern       1
Dublin               1
Ludwigsburg          1
Saarbrücken          1
Name: count, Length: 175, dtype: int64

In [14]:
# Rename same cities naming
filtered_df['location'] = filtered_df['location'].replace({'München': 'Munich'})
filtered_df['location'] = filtered_df['location'].replace({'Köln': 'Cologne'})

In [15]:
# Keep only the specified German cities
specified_cities = ['Berlin', 'Munich', 'Frankfurt', 'Hamburg', 'Cologne']

In [16]:
# Replace cities not in the specified list with "Others"
filtered_df['location'] = filtered_df['location'].apply(lambda x: x if x in specified_cities else 'Others')

In [17]:
# Drop rows where 'location' is "Others"
filtered_df = filtered_df[filtered_df['location'] != 'Others']

In [18]:
# Display the updated value counts for the city column
filtered_df['location'].value_counts()

location
Berlin       1402
Munich        725
Frankfurt     127
Hamburg        90
Cologne        57
Name: count, dtype: int64

# Job Title cleanup

In [19]:
# Inspect job_title column
print(f'Number of job_title missing values: {filtered_df.title.isna().sum()}')

Number of job_title missing values: 0


In [20]:
# Inspect job_title column
filtered_df.title.value_counts()[:30]

title
Software Engineernan            346
nanBackend Developer            199
Backend Developernan            150
nanData Scientist                98
Frontend Developernan            89
Data Scientistnan                86
QA Engineernan                   63
DevOpsnan                        59
nanFullstack Developer           56
nanManager                       52
nanFrontend Developer            51
nanQA                            43
Mobile Developernan              41
Product Managernan               37
nanDevOps                        36
nanMobile Developer              35
ML Engineernan                   34
nanSoftware Architect            33
nanData Engineer                 29
Data Engineernan                 28
nanMachine Learning Engineer     26
Java Developernan                23
nannan                           20
Senior Software Engineernan      15
Designer (UI/UX)nan              15
QAnan                            15
nanBusiness Analyst              13
Engineering Managernan

In [21]:
# Remove "nan" from text in the 'title' column
filtered_df.loc[:, 'title'] = filtered_df['title'].str.replace('nan', '', regex=False)

In [22]:
# Drop rows where 'title' is empty
filtered_df = filtered_df[filtered_df['title'] != '']

In [23]:
# Replace underrepresented positions with "Other"
other_positions = filtered_df.title.value_counts()[(filtered_df.title.value_counts() < 25).values].index
filtered_df.loc[filtered_df.title.isin(other_positions), 'title'] = 'Other'
filtered_df = filtered_df[filtered_df['title'] != 'Other']

In [24]:
# Inspect job_title column
filtered_df.title.value_counts()[:30]

title
Software Engineer            353
Backend Developer            349
Data Scientist               184
Frontend Developer           140
DevOps                        95
Mobile Developer              76
QA Engineer                   63
Fullstack Developer           59
QA                            58
Data Engineer                 57
Manager                       55
Product Manager               46
Software Architect            37
ML Engineer                   34
Machine Learning Engineer     29
Name: count, dtype: int64

# Language used at work cleanup

In [25]:
# Inspect language
print(f'Number of language missing values: {filtered_df.language.isna().sum()}')
filtered_df.language.value_counts()[:6]

Number of language missing values: 13


language
English               1350
Deutsch                143
German                 107
Russian                 18
French                   1
English and German       1
Name: count, dtype: int64

In [26]:
# Replacing "Deutsch" with "German" in the "Main language at work" column
filtered_df['language'] = filtered_df['language'].replace('Deutsch', 'German')

In [27]:
# Fill missing language values with 'English' (assuming missing values are English)
filtered_df.language = filtered_df.language.fillna('English')

In [28]:
# Keep only 'English' and 'German' in the 'language' column
specified_languages = ['English', 'German']
filtered_df['language'] = filtered_df['language'].apply(lambda x: x if x in specified_languages else 'Other')
filtered_df = filtered_df[filtered_df['language'] != 'Other']

In [29]:
filtered_df.language.value_counts()[:6]

language
English    1363
German      250
Name: count, dtype: int64

# Seniority Level cleanup

In [30]:
# Inspect seniority
print(f'Number of level missing values: {filtered_df.level.isna().sum()}')
filtered_df.level.value_counts()

Number of level missing values: 104


level
Senior             820
Middle             425
Lead               144
Junior              85
Head                25
Principal            5
No level             1
intern               1
Working Student      1
student              1
Self employed        1
Name: count, dtype: int64

In [31]:
# Function to determine job level based on job title
def determine_level(row):
    title = row['title']
    if pd.isna(title):  # Check if title is NaN
        return "Middle"  

    title = title.lower()  # Convert to lowercase if title is not NaN

    if "senior" in title:
        return "Senior"
    elif "junior" in title:
        return "Junior"
    elif "intern" in title or "internship" in title:
        return "Intern"
    elif "student" in title or "working student" or "Werkstudent" in title:
        return "Student"
    elif "lead" in title:
        return "Lead"
    elif "head" in title:
        return "Head"
    else:
        return "Middle"

In [32]:
# Applying the function to fill NaN values in 'level' column
filtered_df['level'] = filtered_df.apply(lambda row: determine_level(row) if pd.isnull(row['level']) else row['level'], axis=1)

In [33]:
# Remove slashes and leading/trailing whitespace from the 'level' column
filtered_df['level'] = filtered_df['level'].str.replace('/', '').str.strip()

In [34]:
# Identify the rare values in the 'level' column
rare_levels = filtered_df['level'].value_counts()[6:].index.tolist()

In [35]:
# Replace rare values with 'Other'
filtered_df.loc[filtered_df['level'].isin(rare_levels), 'level'] = 'Other'


In [36]:
# Drop rows where value is "Other"
filtered_df = filtered_df[filtered_df['level'] != 'Other']

In [37]:
filtered_df.level.value_counts()

level
Senior     820
Middle     425
Lead       144
Student    104
Junior      85
Head        25
Name: count, dtype: int64

# Salary Level cleanup

In [38]:
# Inspect seniority
print(f'Number of salary missing values: {filtered_df.salary.isna().sum()}')

Number of salary missing values: 0


In [39]:
filtered_df['salary']

2       88000.0nannan
9       60000.0nannan
10      70000.0nannan
17      60000.0nannan
18      62500.0nannan
            ...      
2999    nan55000.0nan
3000    nan95000.0nan
3001    nan55000.0nan
3004    nan70000.0nan
3008    nan65000.0nan
Name: salary, Length: 1603, dtype: object

In [40]:
# Remove 'nan' variations from the 'salary' column
filtered_df['salary'] = filtered_df['salary'].str.replace(r'nan', '', regex=True)


In [41]:
# Set the display format for float values
pd.options.display.float_format = '{:,.0f}'.format


In [42]:
# Convert the 'salary' column to a numeric data type (float)
filtered_df['salary'] = pd.to_numeric(filtered_df['salary'], errors='coerce')


In [43]:
# Display the top 10 highest salaries
top_10_salaries = filtered_df.nlargest(10, 'salary')
print("Top 10 Highest Salaries:")
print(top_10_salaries)

Top 10 Highest Salaries:
       location               title language   level      salary
2609    Cologne         ML Engineer  English  Middle 500,000,000
2534     Berlin  Frontend Developer  English  Senior     850,000
2332     Munich      Data Scientist  English  Junior     300,000
2560     Berlin    Mobile Developer  English    Lead     240,000
1588     Munich   Backend Developer   German  Senior     200,000
2722     Berlin       Data Engineer  English  Senior     200,000
802      Berlin   Backend Developer  English  Senior     190,000
2250     Berlin             Manager  English    Head     180,000
2593     Munich         ML Engineer   German  Senior     180,000
787   Frankfurt              DevOps   German  Senior     160,000


In [44]:
# Sort the DataFrame by 'salary' in descending order
filtered_df.sort_values(by='salary', ascending=False, inplace=True)


In [45]:
# Drop the first highest values
filtered_df = filtered_df.iloc[15:]


In [46]:
# Reset the index after dropping rows
filtered_df.reset_index(drop=True, inplace=True)


In [47]:
# Display the bottom 10 lowest salaries
bottom_10_salaries = filtered_df.nsmallest(30, 'salary')
print("\nBottom 30 Lowest Salaries:")
print(bottom_10_salaries)


Bottom 30 Lowest Salaries:
       location                title language    level  salary
1586    Hamburg    Backend Developer  English   Middle  10,001
1585     Berlin          ML Engineer  English   Junior  11,500
1584     Berlin    Backend Developer   German   Junior  25,000
1583  Frankfurt    Software Engineer  English   Senior  26,400
1582     Berlin    Software Engineer   German     Lead  28,800
1576    Cologne  Fullstack Developer   German   Junior  30,000
1577     Berlin                   QA  English  Student  30,000
1578     Berlin  Fullstack Developer   German   Middle  30,000
1579     Berlin    Software Engineer   German   Junior  30,000
1580     Berlin      Product Manager  English   Junior  30,000
1581     Berlin      Product Manager  English   Junior  30,000
1575     Berlin                   QA  English  Student  33,000
1574     Berlin                   QA  English   Junior  34,000
1573     Berlin        Data Engineer  English   Middle  34,446
1571     Berlin    Backend 

In [48]:
# Create a copy of the DataFrame to prevent warning
filtered_df = filtered_df.copy()

In [49]:
# Sort the DataFrame by 'salary' in ascending order
filtered_df.sort_values(by='salary', ascending=True, inplace=True)

In [50]:
# Drop the bottom lowest values
filtered_df = filtered_df.iloc[40:]

In [51]:
# Reset the index after dropping rows
filtered_df.reset_index(drop=True, inplace=True)

In [52]:
# Drop missing values in the 'salary' column
filtered_df.dropna(subset=['salary'], inplace=True)

In [53]:
# Display the top 10 highest salaries
top_10_salaries = filtered_df.nlargest(10, 'salary')
print("Top 10 Highest Salaries:")
print(top_10_salaries)

# Display the bottom 10 lowest salaries
bottom_10_salaries = filtered_df.nsmallest(10, 'salary')
print("\nBottom 10 Lowest Salaries:")
print(bottom_10_salaries)

Top 10 Highest Salaries:
     location              title language   level  salary
1542   Berlin    Product Manager  English    Head 150,000
1543   Berlin  Backend Developer  English  Senior 150,000
1544   Berlin  Software Engineer  English  Senior 150,000
1545   Munich   Mobile Developer   German  Senior 150,000
1546   Berlin     Data Scientist  English    Lead 150,000
1539   Berlin     Data Scientist  English  Senior 140,000
1540   Munich  Software Engineer  English    Lead 140,000
1541   Berlin             DevOps  English    Lead 140,000
1537   Berlin  Backend Developer  English    Lead 135,000
1538   Munich     Data Scientist  English  Senior 135,000

Bottom 10 Lowest Salaries:
    location               title language    level  salary
0  Frankfurt  Frontend Developer  English   Middle  44,000
1     Berlin         QA Engineer  English   Middle  45,000
2     Berlin         ML Engineer  English   Junior  45,000
3     Berlin      Data Scientist  English   Junior  45,000
4  Frankfurt  

In [54]:
# Salary thresholds
salary_thresholds = [60000, 80000, 100000]
labels = ['Class 1', 'Class 2', 'Class 3']


# Create a new target column based on the thresholds
def classify_salary(salary):
    for i, threshold in enumerate(salary_thresholds):
        if salary <= threshold:
            return labels[i]
    return labels[-1]  # Assign the last label for salaries above the highest threshold

filtered_df['salary_class'] = filtered_df['salary'].apply(classify_salary)

# Model Training(Classification):

In [55]:
# Split the data into features (X) and the target (y)
X = filtered_df.drop(['salary', 'salary_class'], axis=1)
y = filtered_df['salary_class']  # Use 'salary_class' as the target

In [56]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [57]:
# Initialize the OneHotEncoder
oneh = OneHotEncoder(drop='first', sparse_output=False)

In [58]:
# Fit and transform the categorical features in the training set
X_train = oneh.fit_transform(X_train)

In [59]:
# Transform the categorical features in the test set
X_test= oneh.transform(X_test)

In [60]:
# Create a logistic regression model
reglog = LogisticRegression()
reglog.fit(X_train, y_train)

LogisticRegression()

In [61]:
# Train a Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [62]:
# Train a Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [63]:
# Evaluate the Logistic Regression model
train_score_logistic = reglog.score(X_train, y_train)
test_score_logistic = reglog.score(X_test, y_test)

In [64]:
# Evaluate the Decision Tree model
train_score_decision_tree = decision_tree.score(X_train, y_train)
test_score_decision_tree = decision_tree.score(X_test, y_test)

In [65]:
# Evaluate the Random Forest model
train_score_random_forest = random_forest.score(X_train, y_train)
test_score_random_forest = random_forest.score(X_test, y_test)

In [66]:
print('Logistic Regression:')
print('Score on the train set:', train_score_logistic)
print('Score on the test set:', test_score_logistic)
print('\nDecision Tree:')
print('Score on the train set:', train_score_decision_tree)
print('Score on the test set:', test_score_decision_tree)
print('\nRandom Forest:')
print('Score on the train set:', train_score_random_forest)
print('Score on the test set:', test_score_random_forest)

Logistic Regression:
Score on the train set: 0.6482758620689655
Score on the test set: 0.599483204134367

Decision Tree:
Score on the train set: 0.7137931034482758
Score on the test set: 0.5943152454780362

Random Forest:
Score on the train set: 0.7137931034482758
Score on the test set: 0.5968992248062015


In [67]:
# chosen_model
chosen_model = reglog 

# Save the trained model to a .pkl file
joblib.dump(chosen_model, 'reglog_model.pkl')

['reglog_model.pkl']